In [1]:
import pygame
import pygame.freetype
import sys
import numpy as np
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import time, sys, random, pylab

from math import fabs
from tqdm.notebook import tqdm, trange
from random import randrange
from hrr import *
from IPython.display import clear_output
from sys import argv
from pygame.locals import *

np.set_printoptions(threshold=np.inf)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def create_map():
    tile_rects = []
    for y, layer in enumerate(game_map):
        for x, tile in enumerate(layer):  
            display.blit(dirt_img,(x*16,y*16)) if tile == '1' else tile_rects.append(pygame.Rect(x*16,y*16,16,16))
    return tile_rects

In [3]:
def set_text(goal, current_atr, current_wm, current_signal, val, non_obs):
    display.blit(font.render('Goal (Not available to agent): {0}'.format(goal), False, (0, 0, 0)), (10,20))
    display.blit(font.render('Atr: {0}, Non Obs: {1}'.format(current_atr, non_obs), False, (0, 0, 0)), (10,35))
    display.blit(font.render('WM: {0}'.format(current_wm), False, (0, 0, 0)), (10,50))
    display.blit(font.render('Signal: {0}'.format(current_signal), False, (0, 0, 0)), (10,65))
    display.blit(font.render('Value: {0}'.format(val), False, (0, 0, 0)), (10,80))
    [display.blit(font.render('{0}'.format(i), False, (0, 0, 0)), (20+i*32,142)) for i in range(10)]

In [4]:
def move_game(event, player_rect, position):
    not_done = True
    if event.type == QUIT:
        not_done = False
    player_rect = pygame.Rect(positions[position][0],positions[position][1],5,13)
    return not_done, player_rect

In [5]:
def update():
    screen.blit(pygame.transform.scale(display,window_size),(0,0))
    pygame.display.update()
    clock.tick(fps)

In [6]:
def check_goal(state, goal):
    return False if state == goal else True

In [7]:
def get_moves(state, size_of_maze):
    if(state == 0):
        return size_of_maze - 1, 1
    elif(state == size_of_maze - 1):
        return size_of_maze - 2, 0
    else:
        return state - 1, state + 1

In [8]:
def build_hrr_string(wm, signal, state, atr):
    if wm == "I" and signal == "I":
        return "State:" + str(state) + "*" + "Atr:" + str(atr)
    elif wm == "I":
        return "Signal:" + str(signal) + "*" + "State:" + str(state) + "*" + "Atr:" + str(atr)
    elif signal == "I":
        return "WM:" + str(wm) + "*" + "State:" + str(state) + "*" + "Atr:" + str(atr)
    else:
        return "WM:" + str(wm) + "*" + "Signal:" + str(signal) + "*" + "State:" + str(state) + "*" + "Atr:" + str(atr)

In [9]:
def context_policy_negative(atr):
    return (atr + 1)%num_of_atrs

In [10]:
def context_policy_positive(wm, signal, state, atr):
    val = -9999
    for atr in range(0, num_of_atrs):
        encode_str = build_hrr_string(wm, signal, state, atr)
        temp = np.dot(weights, ltm.encode(encode_str)) + bias
        if temp > val:
            val = temp
            s_atr = atr
    return s_atr

In [11]:
def reward_tkn():
    return "*rewardTkn"

In [12]:
def move_policy(goal, moves, wms, signals, atr, rand_on):
    val = -9999
    for move in moves:
        for wm in list(dict.fromkeys(wms + ["I"])):
            for signal in list(dict.fromkeys(signals + ["I"])):
                if move == goal:
                    encode_str = build_hrr_string(wm, signal, str(move) + reward_tkn(), atr)
                else:
                    encode_str = build_hrr_string(wm, signal, move, atr)
                if (debug):
                    print(encode_str)
                temp = np.dot(weights, ltm.encode(encode_str)) + bias
                if debug:
                    if signal != "I":
                        print("Move: {0}, WM: {1}, Signal: {2}In, Atr: {3}, Value: {4}".format(move, wm, signal, atr, temp))
                    else:
                        print("Move: {0}, WM: {1}, Signal: {2}, Atr: {3}, Value: {4}".format(move, wm, signal, atr, temp))
                if temp > val:
                    val = temp
                    s_move = move
                    if signal != "I":
                        s_wm = signal + "In"
                    else:
                        s_wm = wm
    if(np.random.random_sample() < e_soft) and rand_on:
        if(debug):
            print("RANDOM MOVE")
        return (np.random.choice(moves), wm, atr, True, val)
    
    return (s_move, s_wm, atr, False, val)

In [13]:
def logmod(x):
    return np.sign(x)*np.log(abs(x)+1)

In [14]:
def get_opt_steps(start, goal, size_of_maze):
    opt = abs(goal - start)
    if opt > size_of_maze / 2:
        opt = size_of_maze - opt
    return opt

In [15]:
def start_testing():
    global testing, rand_on, alpha, threshold_alpha, atr_alpha
    testing = True
    rand_on = 0
    alpha = 0.01
    threshold_alpha = 0
    atr_alpha = 0

In [16]:
def reset():
    global num_of_atrs, atr_values, threshold, hrr_length, ltm, weights, eligibility
    num_of_atrs += 1
    atr_values = [1 * reward_good] * num_of_atrs
    if dynamic_threshold:
        threshold = 1
    hrr_length = (num_of_atrs * hrr_length) / (num_of_atrs - 1)
    del ltm
    ltm = LTM(int(hrr_length), normalized)
    weights = hrr(int(hrr_length), normalized)
    eligibility = np.zeros(int(hrr_length))

In [17]:
def accuracy(step_store):
    return (len(step_store)-np.count_nonzero(step_store))*100.0 / len(step_store)

In [18]:
game = {
  "0": {
      "context": 0,
      "goal": 0
  },
  "1": {
      "context": 1,
      "goal": 5
  },
  "2": {
      "context": 2,
      "goal": 8
  },
  "goals": [0, 5, 8]
}

contexts = list(game)[:-1]
goals_list = game['goals']
goals = [[goal] for goal in goals_list]

In [19]:
# Number of training cycles
episodes = 100000

# Hrr parameters
hrr_length = 1024
normalized = True

# How many steps to take before quiting
steps_till_quit = 100

# Task
signals = ["I"]
goals_task = goals

# Maze parameters
size_of_maze = 10
non_obs_task_switch_rate = 500
num_non_obs_tasks = len(goals_task)
num_obs_tasks = len(signals)

# Arguments for neural network
input_size = hrr_length
output_size = 1
discount = 0.7
alpha = 0.1

# Reward for temporal difference learning
reward_bad = -1
reward_good = 0

# Dynamic atrs hyperparameters
num_of_atrs = 1
atr_alpha = 0.0015
atr_values = (np.ones(num_of_atrs) * reward_good).tolist()
atr_threshold = -0.5
threshold_vals = []

# Threshold for non observable task switching
threshold = 0.3
dynamic_threshold = False

# Expolration rate
e_soft = 0.00001
rand_on = 1

# Eligibility trace rate
eli_lambda = 0.0

# Neural network
weights = hrr(hrr_length, normalized)
bias = 1

# Eligibility trace
eligibility = np.zeros(hrr_length)

# Accurcay test percentage
percent_check = 9

# Start values for the agent
non_obs = 0
current_atr = 0
current_wm = "I"
changed = False

# Flag for printing values
debug = False
testing = False

step_store = []

# Ltm is created
ltm = LTM(hrr_length, normalized)

In [20]:
for x in trange(episodes, desc="Episode"):

    current_state = random.randint(0, size_of_maze - 1)
    start = current_state
    current_signal = np.random.choice(signals)
    eligibility *= 0.0
    
    changed = False

    if x%non_obs_task_switch_rate == 0:
        non_obs = (non_obs+1)%len(goals_task)
        changed = True
    if num_obs_tasks == 1:
        goal = goals_task[non_obs][0]
    else:
        goal = goals_task[non_obs][signals.index(current_signal)]

    steps = 0
    opt_steps = get_opt_steps(current_state, goal, size_of_maze)

    if testing == False and x > ((episodes*percent_check) / 10):
        start_testing()
        
    for y in range(steps_till_quit):
        if (current_state == goal):
            encode_str = build_hrr_string(current_wm, current_signal, str(current_state) + reward_tkn(), current_atr)
            goal_hrr = ltm.encode(encode_str)
            goal_value = np.dot(weights, goal_hrr) + bias

            error = reward_good - goal_value
            eligibility *= eli_lambda
            eligibility = eligibility + goal_hrr
            weights = np.add(weights, (alpha * logmod(error) * eligibility))
            
            if dynamic_threshold:
                threshold += threshold_alpha * logmod(error)
                
            atr_values[current_atr] += atr_alpha * logmod(error)            
            break
     
        previous_wm = current_wm
        previous_signal = current_signal
        previous_state = current_state
        previous_atr = current_atr
        
        encode_str = build_hrr_string(previous_wm, previous_signal, previous_state, previous_atr)
        previous_state_hrr = ltm.encode(encode_str)
        previous_value = np.dot(weights, previous_state_hrr) + bias
            
        current_signal = "I"
        left, right = get_moves(previous_state, size_of_maze)
        if previous_signal != "I":
            previous_signal += "In"

        move, wm, current_atr, random_move, val = move_policy(goal, [left, right], [previous_wm, previous_signal], [current_signal], previous_atr, rand_on)
        steps += 1
        current_wm = wm
        current_state = move
        
        if random_move:
            eligibility *= 0.0
            
        if current_state == goal:
            encode_str = build_hrr_string(current_wm, current_signal, str(current_state) + reward_tkn(), current_atr)     
            if debug:
                print("In goal: WM: {1}, ATR: {2}".format(current_wm, current_atr))
        else:
            encode_str = build_hrr_string(current_wm, current_signal, current_state, current_atr)
            
        current_state_hrr = ltm.encode(encode_str)
        current_value = np.dot(weights, current_state_hrr) + bias
        
        sarsa_error = (reward_bad + discount * current_value) - previous_value
        eligibility *= eli_lambda
        eligibility = eligibility + previous_state_hrr
        weights = np.add(weights, (alpha * logmod(sarsa_error) * eligibility))

        atr_values[current_atr] += atr_alpha * logmod(sarsa_error)
        
        if dynamic_threshold:
            threshold += threshold_alpha * logmod(sarsa_error)
        
        if sarsa_error > fabs(threshold) or sarsa_error < -fabs(threshold):
            
            if np.mean(atr_values) < atr_threshold:
                reset()
                
            if sarsa_error > fabs(threshold):
                current_atr = context_policy_positive(current_wm, current_signal, current_state, current_atr)
            elif sarsa_error < -fabs(threshold):
                current_atr = context_policy_negative(previous_atr)
            
            eligibility *= 0.0
            
            if changed:
                steps = 0
                start = current_state
                opt_steps = get_opt_steps(current_state, goal, size_of_maze)
        
    if testing:
        if current_state == goal:
            step_store += [steps - opt_steps]
        else:
            step_store += [steps_till_quit]
            
print(accuracy(step_store))


99.95999599959995


In [21]:
pygame.init()
pygame.font.init()
pygame.display.set_caption("Non-Observable Task")

In [22]:
font = pygame.font.SysFont('arialblack', 12)

In [23]:
not_done = True
not_goal = True

In [24]:
window_size = (672,320)
sky = (128,128,128)

In [25]:
fps = 1000
position = np.random.randint(0,10)

In [26]:
screen = pygame.display.set_mode(window_size,0,32)
display = pygame.Surface((window_size[0]/2,window_size[1]/2)) 
clock = pygame.time.Clock()

In [27]:
dirt_img = pygame.image.load('images/metal.png')
player_img = pygame.image.load('images/player.png').convert()
player_img.set_colorkey((255,255,255))

In [28]:
game_map = [['0'] * 21] * 9
game_map.append(['1'] * 21)

In [29]:
positions = [(23, 130), 
            (55, 130),
            (85, 130),
            (117, 130),
            (149, 130),
            (181, 130),
            (212, 130),
            (245, 130),
            (277, 130),
            (309, 130),]

In [30]:
player_rect = pygame.Rect(positions[position][0],positions[position][1],5,13)

In [31]:
eligibility = np.zeros(int(hrr_length))

non_obs = np.random.choice(contexts)
task = game[str(non_obs)]
goal = task['goal']
context = 0

current_state = position
start = current_state
current_signal = "I"
eligibility *= 0.0

non_obs = 0
current_atr = context
current_wm = "I"
alpha = 0.01
rand_on = 0
eli_lambda = 0.01

changed = False
debug = False
testing = False

steps = 0
val = "None"

In [32]:
while not_done and not_goal:
    display.fill(sky)
    tile_rects = create_map()
    set_text(goal, current_atr, current_wm, current_signal, val, non_obs)

    for event in pygame.event.get():
        if event.type == KEYDOWN and event.key == pygame.K_n:
            update()
            if (current_state == goal):
                encode_str = build_hrr_string(current_wm, current_signal, str(current_state) + reward_tkn(), current_atr)
                goal_hrr = ltm.encode(encode_str)
                goal_value = np.dot(weights, goal_hrr) + bias
                error = reward_good - goal_value
                eligibility *= eli_lambda
                eligibility = eligibility + goal_hrr
                weights = np.add(weights, (alpha * logmod(error) * eligibility))
                atr_values[current_atr] += atr_alpha * logmod(error)    
                not_goal = False
                break
                
            previous_wm = current_wm
            previous_signal = current_signal
            previous_state = current_state
            previous_atr = current_atr
            
            encode_str = build_hrr_string(previous_wm, previous_signal, previous_state, previous_atr)
            previous_state_hrr = ltm.encode(encode_str)
            previous_value = np.dot(weights, previous_state_hrr) + bias

            current_signal = "I"
            left, right = get_moves(previous_state, size_of_maze)
            if previous_signal != "I":
                previous_signal += "In"
                
            move, wm, current_atr, random_move, val = move_policy(goal, [left, right], [previous_wm, previous_signal], [current_signal], previous_atr, rand_on)
            steps += 1
            position = move
            current_wm = wm
            current_state = move
            
            if current_state == goal:
                encode_str = build_hrr_string(current_wm, current_signal, str(current_state) + reward_tkn(), current_atr)
                current_state_hrr = ltm.encode(encode_str)
                val = np.dot(weights, current_state_hrr) + bias
                display.fill(sky)
                tile_rects = create_map()
                update()
                
            set_text(goal, current_atr, current_wm, current_signal, val, non_obs)
            update()
        
            not_done, player_rect = move_game(event, player_rect, position)
            not_goal = check_goal(positions[position], positions[goal])
            
            if current_state == goal:
                encode_str = build_hrr_string(current_wm, current_signal, str(current_state) + reward_tkn(), current_atr)     
            else:
                encode_str = build_hrr_string(current_wm, current_signal, current_state, current_atr)
                
            current_state_hrr = ltm.encode(encode_str)
            current_value = np.dot(weights, current_state_hrr) + bias

            sarsa_error = (reward_bad + discount * current_value) - previous_value
            eligibility *= eli_lambda
            eligibility = eligibility + previous_state_hrr
            weights = np.add(weights, (alpha * logmod(sarsa_error) * eligibility))

            atr_values[current_atr] += atr_alpha * logmod(sarsa_error)
            
            if sarsa_error > fabs(threshold) or sarsa_error < -fabs(threshold):
                if np.mean(atr_values) < atr_threshold:
                    reset()

                if sarsa_error > fabs(threshold):
                    current_atr = context_policy_positive(current_wm, current_signal, current_state, current_atr)
                elif sarsa_error < -fabs(threshold):
                    current_atr = context_policy_negative(previous_atr)

                eligibility *= 0.0

                if changed:
                    steps = 0
                    start = current_state
                    opt_steps = get_opt_steps(current_state, goal, size_of_maze)
            

    display.blit(player_img,(player_rect.x,player_rect.y))
    update()
    if not not_goal:
        display.blit(font.render('You have reached the goal!'.format(goal), False, (0, 0, 0)), (90, 70))
        update()
        time.sleep(2)
pygame.quit()